In [1]:
# Turbo MODEL

from botorch.models.turbo import TurboState
from botorch.optim import optimize_acqf
from botorch.acquisition import qExpectedImprovement
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood

# Initial data
train_X, train_Y = <your small data set>

# Fit initial GP
gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)

# Set up TuRBO state
turbo_state = TurboState(
    dim=train_X.shape[-1],
    batch_size=1,
    length_init=0.8,  # large init trust region
    success_tolerance=3,
    failure_tolerance=3,
)

# Main loop
for iteration in range(MAX_ITERS):
    if turbo_state.restart_triggered:
        turbo_state = TurboState(dim=train_X.shape[-1], batch_size=1)

    # Define acquisition function
    acq = qExpectedImprovement(model=gp, best_f=train_Y.max(), maximize=True)

    # Optimize in trust region
    candidates, _ = optimize_acqf(
        acq_function=acq,
        bounds=turbo_state.get_bounds(train_X),
        q=1,
        num_restarts=5,
        raw_samples=100,
    )

    # Evaluate candidate (real experiment)
    new_Y = objective_function(candidates)

    # Update data
    train_X = torch.cat([train_X, candidates], dim=0)
    train_Y = torch.cat([train_Y, new_Y], dim=0)

    # Update model
    gp.set_train_data(train_X, train_Y, strict=False)
    fit_gpytorch_model(ExactMarginalLogLikelihood(gp.likelihood, gp))

    # Update TuRBO trust region
    turbo_state.update(train_Y[-1])


SyntaxError: invalid syntax (2431776247.py, line 11)